# Video Classifier Using CNN and RNN

In [ ]:
# listing the folder names

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Dataset/train_sample_videos')

label_types = os.listdir('/content/drive/MyDrive/Dataset/train_sample_videos')
print (label_types)  

['metadata.json', 'FAKE', 'REAL']


In [ ]:
dataset_path

['metadata.json', 'FAKE', 'REAL']

# Preparing Training Data

In [ ]:
videos = []

for item in dataset_path:
 name, ext = os.path.splitext(item)
 if ext == '.json':
   continue
  
 # Get all the file names
 all_videos = os.listdir('/content/drive/MyDrive/Dataset/train_sample_videos' + '/' +item)

 # Add them to the list
 for video in all_videos:
    videos.append((item, str('/content/drive/MyDrive/Dataset/train_sample_videos' + '/' +item) + '/' + video))
    
# Build a dataframe        
train_df = pd.DataFrame(data=videos, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


    tag                                         video_name
0  FAKE  /content/drive/MyDrive/Dataset/train_sample_vi...
1  FAKE  /content/drive/MyDrive/Dataset/train_sample_vi...
2  FAKE  /content/drive/MyDrive/Dataset/train_sample_vi...
3  FAKE  /content/drive/MyDrive/Dataset/train_sample_vi...
4  FAKE  /content/drive/MyDrive/Dataset/train_sample_vi...
      tag                                         video_name
295  REAL  /content/drive/MyDrive/Dataset/train_sample_vi...
296  REAL  /content/drive/MyDrive/Dataset/train_sample_vi...
297  REAL  /content/drive/MyDrive/Dataset/train_sample_vi...
298  REAL  /content/drive/MyDrive/Dataset/train_sample_vi...
299  REAL  /content/drive/MyDrive/Dataset/train_sample_vi...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_df.tag.value_counts()


# i had just taken subset of dataset

FAKE    250
REAL     50
Name: tag, dtype: int64

In [ ]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [ ]:
dataset_path = os.listdir('/content/drive/MyDrive/Dataset/test_sample_videos')
print(dataset_path)

categories = os.listdir('/content/drive/MyDrive/Dataset/test_sample_videos')
print("Types of activities found: ", len(dataset_path))

videos = []

for item in dataset_path:
 # Get all the file names
 all_videos = os.listdir('/content/drive/MyDrive/Dataset/test_sample_videos' + '/' +item)

 # Add them to the list
 for video in all_videos:
    videos.append((item, str('/content/drive/MyDrive/Dataset/test_sample_videos' + '/' +item) + '/' + video))
    
# Build a dataframe        
test_df = pd.DataFrame(data=videos, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['REAL', 'FAKE']
Types of activities found:  2
    tag                                         video_name
0  REAL  /content/drive/MyDrive/Dataset/test_sample_vid...
1  REAL  /content/drive/MyDrive/Dataset/test_sample_vid...
2  REAL  /content/drive/MyDrive/Dataset/test_sample_vid...
3  REAL  /content/drive/MyDrive/Dataset/test_sample_vid...
4  REAL  /content/drive/MyDrive/Dataset/test_sample_vid...
     tag                                         video_name
95  FAKE  /content/drive/MyDrive/Dataset/test_sample_vid...
96  FAKE  /content/drive/MyDrive/Dataset/test_sample_vid...
97  FAKE  /content/drive/MyDrive/Dataset/test_sample_vid...
98  FAKE  /content/drive/MyDrive/Dataset/test_sample_vid...
99  FAKE  /content/drive/MyDrive/Dataset/test_sample_vid...


In [ ]:
test_df.tag.value_counts()

FAKE    73
REAL    27
Name: tag, dtype: int64

In [ ]:
train_df.tag.value_counts()

FAKE    250
REAL     50
Name: tag, dtype: int64

In [ ]:
%%capture
!pip install git+https://github.com/tensorflow/docs &> /dev/null
!pip install face_recognition

In [ ]:
# from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths   # for image processing

import matplotlib.pyplot as plt
import tensorflow as tf
# import face_recognition
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [ ]:
# requirements 

%%capture
!pip install git+https://github.com/tensorflow/docs &> /dev/null
!pip install face_recognition

# Data preparation

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 300
Total videos for testing: 100


,Unnamed: 0,video_name,tag
259,259,/content/drive/MyDrive/Dataset/train_sample_vi...,REAL
11,11,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
167,167,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
193,193,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
176,176,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
114,114,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
273,273,/content/drive/MyDrive/Dataset/train_sample_vi...,REAL
106,106,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
54,54,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE
66,66,/content/drive/MyDrive/Dataset/train_sample_vi...,FAKE


# Feed the videos to a network:


In [ ]:
# !pip install google.colab
from google.colab.patches import cv2_imshow

## Extracting Face from the Frames

In [ ]:
# this is the default image which replaces with the empty frame in the video

default_image = cv2.imread('/content/drive/MyDrive/Dataset/train/Fake/emfbhytfhc_5.png')


In [ ]:
IMG_SIZE = 224
SEQ_LENGTH = 20



def crop_face_center(frame):
  face_locations = face_recognition.face_locations(frame)
  # images = []
  if(len(face_locations)>0):
    face_location = face_locations[0]
  else:
    face_locations = face_recognition.face_locations(default_image)
    face_location = face_locations[0]
  top,right,bottom,left = face_location
  face_image = frame[top:bottom, left:right]
  return face_image



def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(total_frames/SEQ_LENGTH),1)
    frames = []
    try:
        for frame_cntr in range(SEQ_LENGTH):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_cntr*skip_frames_window)
            # frameId = cap.get(1)
            ret, frame = cap.read()
            if not ret:
                break
            # if frameId % 2 == 0:
            #   if frameId >37: 
            #     cv2_imshow(frame)
            frame = crop_face_center(frame)
            # print("Frame received")
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            # print("load_video completed")

            # Preventing lengthy video frames
            if len(frames) == max_frames:
                break
        print("Completed extracting frames")
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction  through InceptionV3

In [ ]:
IMG_SHAPE = (224,224,3)
IMG_SIZE = 224
# Function to extract features from Video Frames
def build_feature_extractor():

    # Using pretrained EfficientNetB1 Model
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )

    # Adding a Preprocessing layer in the Model
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
feature_extractor.save('/content/drive/MyDrive/models/CNN_RNN/feature_extractor.h5')

### Label Encoding
StringLookup layer encode the class labels as integers.

In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()

['FAKE', 'REAL']


Finally, we can put all the pieces together to create our data processing utility.

In [ ]:
#print(train_data[0].shape)
#train_data[0]

In [ ]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
# this function takes high time approx. 5 hrs for 300 videos trainning.


def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    count = 0
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()
    
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            
            print("video length: and no. of videos: ",video_length,count)
            count = count +1
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df,'')
test_data, test_labels = prepare_all_videos(test_df, '')

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")
print(f"test_labels in train set: {test_labels.shape}")


Completed extracting frames
video length: and no. of videos:  20 0
1/1 [==============================] - 0s 156ms/step
Completed extracting frames
video length: and no. of videos:  20 1
1/1 [==============================] - 0s 159ms/step
Completed extracting frames
video length: and no. of videos:  20 2
1/1 [==============================] - 0s 177ms/step
Completed extracting frames
video length: and no. of videos:  20 3
1/1 [==============================] - 0s 157ms/step
Completed extracting frames
video length: and no. of videos:  20 4
1/1 [==============================] - 0s 152ms/step
Completed extracting frames
video length: and no. of videos:  20 5
1/1 [==============================] - 0s 173ms/step
Completed extracting frames
video length: and no. of videos:  20 6
1/1 [==============================] - 0s 166ms/step
Completed extracting frames
video length: and no. of videos:  20 7
1/1 [==============================] - 0s 141ms/step
Completed extracting frames
video length

In [ ]:
with open('/models', 'wb') as f:
    np.savez(f, train_data[0],train_data[1])
with open('/models', 'wb') as f:
    np.savez(f, test_data[0],test_data[1])

# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)
    # opt = keras.layers.optimizers.Adam(learning_rate=0.0001)
    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="Adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.25,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([train_data[0], train_data[1]], train_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
4/4 [==============================] - ETA: 0s - loss: 0.5783 - accuracy: 0.6638
Epoch 1: val_loss improved from inf to 1.54623, saving model to ./tmp\video_classifier
4/4 [==============================] - 30s 3s/step - loss: 0.5783 - accuracy: 0.6638 - val_loss: 1.5462 - val_accuracy: 0.0000e+00
Epoch 2/30
4/4 [==============================] - ETA: 0s - loss: 0.5797 - accuracy: 0.7328
Epoch 2: val_loss did not improve from 1.54623
4/4 [==============================] - 1s 244ms/step - loss: 0.5797 - accuracy: 0.7328 - val_loss: 1.5778 - val_accuracy: 0.0000e+00
Epoch 3/30
4/4 [==============================] - ETA: 0s - loss: 0.5595 - accuracy: 0.7845
Epoch 3: val_loss improved from 1.54623 to 1.53971, saving model to ./tmp\video_classifier
4/4 [==============================] - 1s 277ms/step - loss: 0.5595 - accuracy: 0.7845 - val_loss: 1.5397 - val_accuracy: 0.0000e+00
Epoch 4/30
4/4 [==============================] - ETA: 0s - loss: 0.5407 - accuracy: 0.7759
Epoch 4: v

# Saving my Model

In [ ]:
sequence_model.save("models/CNN_RNN/CNN_RNN.h5")

## Loading the Model

In [ ]:
from tensorflow.keras.models import load_model
seq_model = load_model('/content/drive/MyDrive/models/CNN_RNN/CNN_RNN.h5')
sequence_model = seq_model

In [ ]:


def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(path)
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return probabilities

In [ ]:
# for idx, path in enumerate(video_paths):
#   class_vocab = label_processor.get_vocabulary()
def predict_test():
  test_pred = pd.DataFrame(columns=["video_name","pred","tag"])
  video_paths = test_df["video_name"].values.tolist()
  class_vocab = label_processor.get_vocabulary()
  tag_set = test_df["tag"].values.tolist()
  for (tag, path) in zip(tag_set,video_paths):
    f_name = os.path.basename(path)
    tag_num = 0 if tag=='FAKE' else 1
    prob = sequence_prediction(path)
    pred = prob.argmax()
    print(class_vocab[pred])
    test_pred = test_pred.append({'video_name':f_name, 'pred':pred, 'tag':tag_num}, ignore_index=True)
  # display(test_pred)
  return test_pred

In [ ]:
pred_res = predict_test()

Completed extracting frames
1/1 [==============================] - 2s 2s/step
  FAKE: 56.28%
  REAL: 43.72%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 19ms/step
  FAKE: 65.23%
  REAL: 34.77%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 20ms/step
  FAKE: 66.34%
  REAL: 33.66%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 22ms/step
  FAKE: 62.18%
  REAL: 37.82%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 19ms/step
  FAKE: 58.76%
  REAL: 41.24%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 22ms/step
  FAKE: 66.02%
  REAL: 33.98%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 22ms/step
  FAKE: 60.70%
  REAL: 39.30%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 19ms/step
  FAKE: 61.62%
  REAL: 38.38%
FAKE
Completed extracting frames
1/1 [==============================] - 0s 21ms/step
  

In [ ]:
pred_res.to_csv('pred_res.csv')
!cp pred_res.csv "/content/drive/MyDrive/variables/"

### Test Video

In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/test/FAKE/bjsmaqefoi.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")

In [ ]:
test_video = "Data/test/FAKE/bjsmaqefoi.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/test/FAKE/bjsmaqefoi.mp4
Completed extracting frames
1/1 [==============================] - 0s 88ms/step
  FAKE: 90.97%
  REAL:  9.03%


In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/test/FAKE/bgmlwsoamc.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")


In [ ]:
test_video = "Data/test/FAKE/bgmlwsoamc.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/test/FAKE/bgmlwsoamc.mp4
Completed extracting frames
1/1 [==============================] - 0s 68ms/step
  REAL: 51.04%
  FAKE: 48.96%


In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/train/FAKE/aagfhgtpmv.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")

In [ ]:
test_video = "Data/train/FAKE/aagfhgtpmv.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/train/FAKE/aagfhgtpmv.mp4
Completed extracting frames
1/1 [==============================] - 0s 76ms/step
  FAKE: 83.73%
  REAL: 16.27%


## Real Video Predictions

In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/test/REAL/cppdvdejkc.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")


In [ ]:
test_video = "Data/test/REAL/cppdvdejkc.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/test/REAL/cppdvdejkc.mp4
Completed extracting frames
1/1 [==============================] - 0s 72ms/step
  REAL: 68.67%
  FAKE: 31.33%


In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/test/REAL/dkzvdrzcnr.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")

In [ ]:
test_video = "Data/test/REAL/dkzvdrzcnr.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/test/REAL/dkzvdrzcnr.mp4
Completed extracting frames
1/1 [==============================] - 0s 65ms/step
  REAL: 80.11%
  FAKE: 19.89%


In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/test/REAL/duycddgtrl.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")

In [ ]:
test_video = "Data/test/REAL/duycddgtrl.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/test/REAL/duycddgtrl.mp4
Completed extracting frames
1/1 [==============================] - 0s 64ms/step
  REAL: 60.56%
  FAKE: 39.44%


In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="Data/test/REAL/dhxctgyoqj.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")

In [ ]:
test_video = "Data/test/REAL/dhxctgyoqj.mp4"
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: Data/test/REAL/dhxctgyoqj.mp4
Completed extracting frames
1/1 [==============================] - 0s 83ms/step
  FAKE: 65.13%
  REAL: 34.87%
